# **1. Memasukkan Data dan Melihat Jumlah Alternatif & Kriteria**

In [9]:
#Import file CSV
import pandas as pd

df = pd.read_csv('cabe.csv')
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 13 columns):
 #   Column                                                    Non-Null Count  Dtype 
---  ------                                                    --------------  ----- 
 0   Alternatif                                                6 non-null      object
 1   Daun Mengalami Kelayuan                                   6 non-null      object
 2   Daun Menguning dan Menjalar ke Ranting                    6 non-null      object
 3   Warna Jaringan Akar dan Batang menjadi Coklat             6 non-null      object
 4   Layu secara tiba-tiba                                     6 non-null      object
 5   Semua Daun Layu tetapi Tidak Berubah Warnanya             6 non-null      object
 6   Jaringan Veskuler dari Batang Bagian Bawah Akar Coklat    6 non-null      object
 7   Pucuk Daun Berubah Menjadi Kuning Jelas                   6 non-null      object
 8   Tulang Daun Menebal dan Daun Mengg

In [199]:
#Mencari jumlah Kolom
count_columns = len(df.columns)
print('jumlah kolom: ' + str(count_columns))

#Mencari jumlah Baris
count_rows = len(df)
print('jumlah baris: ' + str(count_rows))

jumlah kolom: 13
jumlah baris: 6


# **2. Matriks Keputusan Ternormalisasi**

In [200]:
#Membuang baris Kategori & Bobot dan kolom Alternatif
df2 = df.drop([count_rows-2, count_rows-1])
df2 = df2.drop('Alternatif', axis='columns').astype(int)

#Tiap-tiap bobot dikuadratkan
df2_square = df2**2

#Menjumlahkan kuadrat dari setiap kriteria dan diakar
sqrt = df2_square.sum()**0.5
sqrt

Daun Mengalami Kelayuan                                     7.141428
Daun Menguning dan Menjalar ke Ranting                      8.124038
Warna Jaringan Akar dan Batang menjadi Coklat               8.000000
Layu secara tiba-tiba                                       8.124038
Semua Daun Layu tetapi Tidak Berubah Warnanya               7.681146
Jaringan Veskuler dari Batang Bagian Bawah Akar Coklat      7.071068
Pucuk Daun Berubah Menjadi Kuning Jelas                     7.071068
Tulang Daun Menebal dan Daun Menggulung Ke Atas             6.708204
Tanaman Kerdil dan Tidak Berbuah                            6.708204
Muncul bercak bulat berwarna coklat pada Daun dan Kering    7.071068
Bercak berwarna pucat putih dengan warna tepi lebih tua     6.557439
Terdapat Lubang pada bercak tua                             6.708204
dtype: float64

In [201]:
#Menghitung nilai R
df2_norm = df2 / sqrt
df2_norm

,Daun Mengalami Kelayuan,Daun Menguning dan Menjalar ke Ranting,Warna Jaringan Akar dan Batang menjadi Coklat,Layu secara tiba-tiba,Semua Daun Layu tetapi Tidak Berubah Warnanya,Jaringan Veskuler dari Batang Bagian Bawah Akar Coklat,Pucuk Daun Berubah Menjadi Kuning Jelas,Tulang Daun Menebal dan Daun Menggulung Ke Atas,Tanaman Kerdil dan Tidak Berbuah,Muncul bercak bulat berwarna coklat pada Daun dan Kering,Bercak berwarna pucat putih dengan warna tepi lebih tua,Terdapat Lubang pada bercak tua
0,0.700140,0.615457,0.5,0.492366,0.520756,0.424264,0.424264,0.596285,0.596285,0.424264,0.457496,0.596285
1,0.140028,0.492366,0.5,0.615457,0.650945,0.565685,0.424264,0.447214,0.298142,0.424264,0.457496,0.298142
2,0.560112,0.492366,0.5,0.369274,0.390567,0.565685,0.565685,0.596285,0.596285,0.565685,0.457496,0.447214
3,0.420084,0.369274,0.5,0.492366,0.390567,0.424264,0.565685,0.298142,0.447214,0.565685,0.609994,0.596285


# **3. Menentukan Matriks Keputusan Ternormalisasi Terbobot**

In [202]:
#Menghitung nilai Y
df2_weight = df2_norm * df.iloc[-1,1:count_columns].astype(int)
df2_weight

,Daun Mengalami Kelayuan,Daun Menguning dan Menjalar ke Ranting,Warna Jaringan Akar dan Batang menjadi Coklat,Layu secara tiba-tiba,Semua Daun Layu tetapi Tidak Berubah Warnanya,Jaringan Veskuler dari Batang Bagian Bawah Akar Coklat,Pucuk Daun Berubah Menjadi Kuning Jelas,Tulang Daun Menebal dan Daun Menggulung Ke Atas,Tanaman Kerdil dan Tidak Berbuah,Muncul bercak bulat berwarna coklat pada Daun dan Kering,Bercak berwarna pucat putih dengan warna tepi lebih tua,Terdapat Lubang pada bercak tua
0,3.50070,2.461830,2.0,1.969464,2.603778,2.121320,2.121320,2.981424,2.385139,1.697056,1.829983,2.981424
1,0.70014,1.969464,2.0,2.461830,3.254723,2.828427,2.121320,2.236068,1.192570,1.697056,1.829983,1.490712
2,2.80056,1.969464,2.0,1.477098,1.952834,2.828427,2.828427,2.981424,2.385139,2.262742,1.829983,2.236068
3,2.10042,1.477098,2.0,1.969464,1.952834,2.121320,2.828427,1.490712,1.788854,2.262742,2.439977,2.981424


# **4. Menentukan matriks solusi ideal positif & matriks solusi ideal negatif**

In [204]:
A_plus_minus = {
    'A_plus'  : [],
    'A_minus' : []
}

for column in range(1, count_columns):

  #Kondisi jika kategori dari kriteria adalah benefit
  if df.iloc[-2,column] == 'benefit':
    A_plus_minus['A_plus'].append(df2_weight.iloc[:,column-1].max())
    A_plus_minus['A_minus'].append(df2_weight.iloc[:,column-1].mmin())

  #Kondisi jika kategori dari kriteria adalah cost  
  else:
    A_plus_minus['A_plus'].append(df2_weight.iloc[:,column-1].min())
    A_plus_minus['A_minus'].append(df2_weight.iloc[:,column-1].max())

print(A_plus_minus['A_plus'])
print(A_plus_minus['A_minus'])

[0.7001400420140048, 1.4770978917519928, 2.0, 1.4770978917519928, 1.952833664712358, 2.1213203435596424, 2.1213203435596424, 1.4907119849998598, 1.1925695879998879, 1.697056274847714, 1.829982843991256, 1.4907119849998598]
[3.500700210070024, 2.4618298195866544, 2.0, 2.4618298195866544, 3.254722774520597, 2.82842712474619, 2.82842712474619, 2.9814239699997196, 2.3851391759997758, 2.262741699796952, 2.4399771253216747, 2.9814239699997196]


# **5. Menentukan Jarak Antara Nilai Setiap Alternatif dengan Matriks Solusi Ideal Positif dan Matriks Solusi Ideal Negatif**

In [210]:
#Mencari nilai D+ dan D-
D_plus_minus = {
    'D_plus'  : [],
    'D_minus' : []
}

D_plus_square = (df2_weight - A_plus_minus['A_plus'])**2
D_minus_square = (df2_weight - A_plus_minus['A_minus'])**2

for row in range(count_rows-2):
  D_plus = (D_plus_square.loc[row].sum())**0.5
  D_minus = (D_minus_square.loc[row].sum())**0.5

  D_plus_minus['D_plus'].append(D_plus)
  D_plus_minus['D_minus'].append(D_minus)

print(D_plus_minus['D_plus'])
print(D_plus_minus['D_minus'])

[3.917352926052127, 1.9906260376861487, 3.189700448052543, 2.44398841183491]
[1.53565819088734, 3.671192520228545, 2.079634853430102, 2.8187938125839866]


# **6. Mencari nilai preferensi untuk setiap alternatif**

In [206]:
#Mencari nilai V
total = []
for row in range(count_rows-2):
  v = D_plus_minus['D_minus'][row] / (D_plus_minus['D_minus'][row] + D_plus_minus['D_plus'][row])
  total.append(v)
total

[0.2816165523882586,
 0.6484122517660974,
 0.39466739815265706,
 0.5356090547514971]

In [207]:
#Memberikan ranking untuk setiap alternatif
df3 = df[['Alternatif']]
df3 = df3.drop([count_rows-2, count_rows-1])
df3.insert(1, 'Total', total)

ranking = df3.iloc[:,1].rank(ascending=False)
df3.insert(2, 'Ranking', ranking)
df3

,Alternatif,Total,Ranking
0,Penyakit Layu Bakteri Ralstonia (Ralstonia Sol...,0.281617,4.0
1,Layu Fuserium (Fuserium Oxysporum Sp),0.648412,1.0
2,Penyakit Virus Kuning (Gemini Virus),0.394667,3.0
3,Penyakit Bercak Daun (Cercospora sp.),0.535609,2.0
